### Importing images and creating an Xarray Dataset

In [46]:
import rasterio as rio
import numpy as np
import scipy.ndimage
from glob import glob

In [40]:
img_files = glob('data/S2B_*.tif')

In [41]:
img_files = sorted(img_files)
img_files

['data/S2B_20PRQ_20190730_0_L2A_B01.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B02.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B03.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B04.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B05.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B06.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B07.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B08.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B09.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B11.tif',
 'data/S2B_20PRQ_20190730_0_L2A_B12.tif']

In [42]:
bands = [file.replace('data/S2B_20PRQ_20190730_0_L2A_', '').replace('.tif', '') for file in img_files]
bands

['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12']

In [19]:
def list_band_properties(img_path):
  img = rio.open(img_path)
  name = img.name
  width, height = img.shape
  left, bottom, right, top = img.bounds
  width_projected_units = right - left
  height_projected_units = top - bottom
  resolution_x = width_projected_units / width
  resolution_y = height_projected_units / height
  print(f'Name: {name}')
  print(f'Width, Height (pixels): {width}, {height}')
  print(f'Pixel size (X, Y): {resolution_x}, {resolution_y}')


In [43]:
images = {}


for i, file in enumerate(img_files):
  img = rio.open(file)
  band_name = bands[i]
  width, height = img.shape
  left, bottom, right, top = img.bounds
  width_projected_units = right - left
  height_projected_units = top - bottom
  pixel_size = width_projected_units / width
  images[band_name] = {
    'band': band_name,
    'data': img.read(),
    'pixel_size': pixel_size,
    'width': width,
    'height': height,
    'transform': img.transform,
    'crs': img.crs
  }

In [59]:

def get_band(band_name):
  if not band_name in bands:
    raise Exception(f'Invalid band name: {band_name}')
  return images[band_name]

def normalized_difference(band_1='B04', band_2='B03', index_name='NDVI'):
  band_1 = get_band(band_1)
  band_2 = get_band(band_2)

  band_1_data = band_1['data']
  band_2_data = band_2['data']

  band_1_width= band_1['width']
  band_2_width = band_2['width']

  result_properties =  {
    'pixel_size': band_1['pixel_size'],
    'width': band_1['width'],
    'height': band_1['height'],
    'transform': band_1['transform'],
    'crs': band_1['crs']
  }

  if band_1_width > band_2_width:
    result_properties =  {
      'pixel_size': band_2['pixel_size'],
      'width': band_2['width'],
      'height': band_2['height'],
      'transform': band_2['transform'],
      'crs': band_2['crs']
    }
    zoom = int(band_1_width / band_2_width)
    band_2_data = scipy.ndimage.zoom(band_2_data, zoom=zoom, order=0)
    
  elif band_2_width > band_1_width:
    result_properties =  {
      'pixel_size': band_1['pixel_size'],
      'width': band_1['width'],
      'height': band_1['height'],
      'transform': band_1['transform'],
      'crs': band_1['crs']
    }
    zoom = int(band_2_width / band_1_width)
    band_1_data = scipy.ndimage.zoom(band_1_data, zoom=zoom, order=0)

  print(f'band 1: {band_1_data.shape}')
  print(f'band 2: {band_2_data.shape}')

  result_properties['name'] = index_name
  result_properties['data'] = (band_1_data - band_2_data) / (band_1_data + band_2_data)
  return result_properties

def divide(band_numerator, band_denominator, index_name=''):
  numerator = get_band(band_numerator)
  denominator = get_band(band_denominator)

  numerator_array = numerator['data']
  denominator_array = denominator['data']

  result_properties =  {
    'pixel_size': numerator['pixel_size'],
    'width': numerator['width'],
    'height': numerator['height'],
    'transform': numerator['transform'],
    'crs': numerator['crs']
  }

  if numerator['width'] > denominator['width']:
    result_properties =  {
      'pixel_size': denominator['pixel_size'],
      'width': denominator['width'],
      'height': denominator['height'],
      'transform': denominator['transform'],
      'crs': denominator['crs']
    }
    zoom  = int(numerator['width'] / denominator['width'])
    denominator_array = scipy.ndimage.zoom(denominator_array, zoom=zoom, order=0)
  elif numerator['width'] < denominator['width']:
    result_properties =  {
      'pixel_size': numerator['pixel_size'],
      'width': numerator['width'],
      'height': numerator['height'],
      'transform': numerator['transform'],
      'crs': numerator['crs']
    }
    zoom  = int(denominator['width'] / numerator['width'])
    numerator_array = scipy.ndimage.zoom(numerator_array, zoom=zoom, order=0)

  result_properties['name'] = index_name
  result_properties['data'] = numerator_array / denominator_array
  
  return result_properties

In [60]:
# NDVI - normalized difference vegetation index
ndvi = normalized_difference('B08', 'B04', 'NDVI')
# NDMI - normalized difference mangrove index
ndmi = normalized_difference('B03', 'B12', 'NDMI')
# MNDWI - modified normalized difference water index
mndwi = normalized_difference('B03', 'B11', 'MNDWI')
# SR - simple ratio
sr = divide('B08', 'B04', 'SR')
# R54 - band ratio 54
ratio_54 = divide('B11', 'B08', 'R54')
# R35 - band ratio 35
ratio_35 = divide('B04', 'B11', 'R35')
# GCVI
gcvi = divide('B08', 'B03', 'GCVI')
gcvi['data'] = gcvi['data'] - np.ones_like(gcvi['data'])

generated_images = [ndvi, ndmi, mndwi, sr, ratio_54, ratio_35, gcvi]

for image in generated_images:
  name = image['name']
  images[name] = image

band 1: (1, 10980, 10980)
band 2: (1, 10980, 10980)


/var/folders/x1/f8wprbyd6yn_gbw19z35hkdr0000gn/T/ipykernel_20025/4241675620.py:50: RuntimeWarning: invalid value encountered in divide
  result_properties['data'] = (band_1_data - band_2_data) / (band_1_data + band_2_data)


band 1: (1, 10980, 10980)
band 2: (2, 10980, 10980)
band 1: (1, 10980, 10980)
band 2: (2, 10980, 10980)


/var/folders/x1/f8wprbyd6yn_gbw19z35hkdr0000gn/T/ipykernel_20025/4241675620.py:90: RuntimeWarning: invalid value encountered in divide
  result_properties['data'] = numerator_array / denominator_array


In [61]:
images

{'B01': {'band': 'B01',
  'data': array([[[   0,    0,    0, ...,  373,  395,  295],
          [   0,    0,    0, ...,  517,  544,  396],
          [   0,    0,    0, ...,  513,  544,  390],
          ...,
          [   0,    0,    0, ..., 2324, 2910, 2322],
          [   0,    0,    0, ..., 5029, 5812, 4838],
          [   0,    0,    0, ..., 5544, 6292, 5538]]], dtype=uint16),
  'pixel_size': 60.0,
  'width': 1830,
  'height': 1830,
  'transform': Affine(60.0, 0.0, 799980.0,
         0.0, -60.0, 1000020.0),
  'crs': CRS.from_epsg(32620)},
 'B02': {'band': 'B02',
  'data': array([[[   0,    0,    0, ...,  429,  451,  411],
          [   0,    0,    0, ...,  441,  395,  428],
          [   0,    0,    0, ...,  408,  423,  408],
          ...,
          [   0,    0,    0, ..., 6112, 6020, 6064],
          [   0,    0,    0, ..., 6300, 6196, 6264],
          [   0,    0,    0, ..., 6500, 6324, 6420]]], dtype=uint16),
  'pixel_size': 10.0,
  'width': 10980,
  'height': 10980,
  'transform

In [32]:
img.count

1